In [613]:
import sqlite3
import pandas as pd
import numpy as np
import global_imports
from src import xbox_spreadsheet, origin_parse, report, write_to_excel
import re
import json

# Not necessary in a py file as jupyter does not set the path correctly
import os
if os.getcwd().split('\\')[-1] != global_imports.root_folder:
    os.chdir(global_imports.root_folder)

In [614]:
def import_database_from_sql(sql_query,db_path):
    connector = sqlite3.connect(db_path)
    db = pd.read_sql(sql_query, connector)
    return db

game_piece_type_query = 'SELECT * FROM GamePieceTypes'
game_piece_type = import_database_from_sql(game_piece_type_query, global_imports.main_db_path)

all_db_query = """SELECT GamePieces.releaseKey, GamePieces.gamePieceTypeId, GamePieces.value
    FROM GameLinks
	JOIN GamePieces ON GameLinks.releaseKey = GamePieces.releaseKey
	JOIN LibraryReleases ON GameLinks.releaseKey = LibraryReleases.releaseKey"""
main_db = import_database_from_sql(all_db_query, global_imports.main_db_path)

In [615]:
column_names = ['title', 'meta']

def split_metadata_into_columns(column_list, db):
    """Splits metadata of the main_db into different rows. Arguments are a list of column_names
    and the dataframe"""

# Creates columns for each item in column_list
    for val in column_list:
        type_id = int(game_piece_type.loc[game_piece_type['type'] == val, 'id'])
        db[val] = db.loc[db['gamePieceTypeId'] == type_id, 'value']

# Drop rows that are null in all of the generated columns. This will remove every row that
# is not in column_list
    db = db.dropna(subset=column_list, how='all')

# Drop every other row we do not need anymore
    db = db.drop(['value','gamePieceTypeId'], axis=1)

# Group by releaseKey since the data is in its own rows. Fill row with NAN if every row is NAN
    db = db.groupby('releaseKey', as_index=True).agg(lambda x: np.nan if x.isnull().all()
    else x.dropna()).reset_index()
    db = db.set_index(np.arange(0, len(db)))
    return db


main_db = split_metadata_into_columns(column_names, main_db)

In [616]:
def extract_date(db):
    """Extract Release Date from metadata and convert it into MM YYYY format"""

    date_pattern = 'releaseDate\":(\d{9,10})'
    def format_date(x):
        date = re.search(date_pattern, x)
        if date:
            val = pd.to_datetime(date.group(1), unit='s')
            val = val.strftime('%Y %b')
            return val
        else:
            return 'No Date'

    db['date'] = db['meta'].apply(format_date)
    db = db.drop('meta', axis=1)
    return db

main_db = extract_date(main_db)

In [617]:
def create_platform(db):
    with open(global_imports.platforms_json) as platform_file:
        platform = json.load(platform_file)

    # Create a regex pattern of all platforms to match and then return the actual platform name
    platform_keys = list(platform.keys())
    platform_pattern = re.compile(r"(\b{}\b)".format("|".join(platform_keys)))

    def platform_extract(x):
        m = platform_pattern.match(x)
        if m:
            return platform[m.group(1)]

    db['platform'] = db['releaseKey'].apply(platform_extract)
    return db

main_db = create_platform(main_db)

In [618]:
def remove_hidden_games_by_gog(db):
    hidden_db_query = 'SELECT releaseKey, isDlc, isVisibleInLibrary FROM ReleaseProperties'
    hidden_db = import_database_from_sql(hidden_db_query, global_imports.main_db_path)

    # Delete everything marked as DLC or not visible in library
    db = db[~db['releaseKey'].isin(hidden_db.loc[hidden_db['isVisibleInLibrary'] == 0, 'releaseKey'])]
    db = db[~db['releaseKey'].isin(hidden_db.loc[hidden_db['isDlc'] == 1, 'releaseKey'])]
    return db

main_db = remove_hidden_games_by_gog(main_db)

In [619]:
def remove_manual_hidden_games_by_user(db):
    user_hidden_db_query = 'SELECT releaseKey, isHidden FROM UserReleaseProperties'
    user_hidden_db = import_database_from_sql(user_hidden_db_query, global_imports.main_db_path)

    # Remove games in main_db but not in user_hidden_db as these were never owned
    db = db.drop(db[~db['releaseKey'].isin(user_hidden_db['releaseKey'])].index)
    db = db[~db['releaseKey'].isin(user_hidden_db.loc[user_hidden_db['isHidden'] == 1, 'releaseKey'])]
    db = db.reset_index(drop=True)
    return db

main_db = remove_manual_hidden_games_by_user(main_db)

In [620]:
# Remove brackets and quotes around titlee
main_db['title'] = main_db['title'].apply(lambda x: x[10:-2])

def format_titles(db, column):

    """Format Title by removing special apostrophe"""

    # Remove special apostrophe
    db[column] = db[column].str.replace('’', '')

    # Remove the at the beginning of the word
    db[column] = db[column].str.replace('^The\s', '')

    """Format Title by removing trademark and copyright symbols"""
    # Remove Trademark and Copyright Symbols
    db[column] = db[column].str.replace('\u2122|\u00AE','')
    return db

main_db = format_titles(main_db, 'title')

In [621]:
def remove_duplicates(db):

    # Delete rows with same title and platform
    db = db.drop_duplicates(subset=['title','platform'], keep='last').reset_index(drop=True)

    # Delete rows with same title in 'Other' Platform
    dup = db[db.duplicated(subset=['title'], keep=False)].sort_values('title')
    db = db.drop(dup[dup['platform'].str.match('Other')].index).reset_index(drop=True)
    db = db[~db['title'].str.contains('trial$',flags=re.IGNORECASE)]
    return db
main_db = remove_duplicates(main_db)

In [622]:
def create_tag(db):

    # Import tags database
    tag_db_query = 'SELECT releaseKey, tag FROM UserReleaseTags'
    tag_db = import_database_from_sql(tag_db_query, global_imports.main_db_path)
    temp = db.merge(tag_db, how='left',left_on=['releaseKey'],right_on=['releaseKey'])

    def create_status(x):
        pattern = re.compile(r"S - (\w+)")
        if pattern.match(x):
            return pattern.match(x).group(1)
        return np.nan

    def create_length(x):
        pattern = re.compile(r"L - (\w+)")
        if pattern.match(x):
            return pattern.match(x).group(1)
        return np.nan

    def create_platform_other(x):
        pattern = re.compile(r"P - (\w+)")
        if pattern.match(x):
            return pattern.match(x).group(1)
        return np.nan

    # Create Status and Length as two separate Columns
    temp['tag'] = temp['tag'].fillna('No tag')
    temp['Status'] = temp['tag'].apply(create_status)
    temp['Length'] = temp['tag'].apply(create_length)
    temp['other_platform'] = temp['tag'].apply(create_platform_other)
    temp.drop(['tag'], axis=1, inplace=True)
    temp = temp.groupby('releaseKey')[['Status','Length', 'other_platform']].first().reset_index()
    db = db.merge(temp, how='left', on='releaseKey')
    return db

# Add the Status and Length to the main Table
main_db = create_tag(main_db)

In [623]:
def platform_tag_correction(db):

    db.loc[db['platform'] == 'Other', 'platform'] = np.NaN
    db.loc[db['platform'] == 'Xbox Gamepass', 'platform'] = np.NaN
    db['platform'] = db['platform'].combine_first(db['other_platform'])
    db.drop(['other_platform'], axis=1, inplace=True)
    return db

main_db = platform_tag_correction(main_db)

In [624]:
main_db = main_db.rename(columns={"title": "Title", "date": "Release", "platform": "Platform"})
main_db.reset_index(drop = True, inplace = True)

In [625]:
def group_platform(db):
    temp = db.groupby('Title').agg({'Platform':'; '.join})
    db = db.merge(temp, how='inner',left_on=['Title'],right_on=['Title'])
    db = db.drop('Platform_x', axis=1)
    db = db.drop_duplicates(subset=['Title', 'Platform_y'], keep='last')
    db = db.reset_index(drop=True)
    db = db.rename(columns={"Platform_y": "Platform"})
    return db

main_db = group_platform(main_db)

In [626]:
def add_gameplay_time(db):
    time_query = 'SELECT ProductName, StatTotalFullRuntime FROM Applications'
    time_db = import_database_from_sql(time_query, global_imports.gtt_db_path)
    db = db.merge(time_db, how='left', left_on='Title', right_on='ProductName')
    db = db.drop('ProductName', axis=1)
    db['StatTotalFullRuntime'] = db['StatTotalFullRuntime'].fillna(0)
    db = db.rename(columns={"StatTotalFullRuntime": "GameplayTime"})

    def convert_to_hours(x):
        if x:

            # The value of gameplay has additional zeroes. Remove zeroes and convert to minutes
            x = x/(10000000*60)
            hours = int(x/60)
            minutes = int(x%60)

            # Return output according to whether minutes or hours are zero
            if hours and minutes:
                return str(hours) + ' Hours ' + str(minutes) + ' Minutes'
            elif hours:
                return str(hours) + ' Hours'
            elif minutes:
                return str(minutes) + ' Minutes'
        else:
            return 0
    db['GameplayTime'] = db['GameplayTime'].apply(convert_to_hours)
    return db

main_db = add_gameplay_time(main_db)

In [627]:
main_db


,releaseKey,Title,Release,Status,Length,Platform,GameplayTime
0,battlenet_prometheus,Overwatch,2016 May,Completed,Infinite,Battle.net,137 Hours 47 Minutes
1,epic_11e598b192324994abce05bad4f81b50,A Total War Saga: TROY,2020 Aug,Backlog,Infinite,Epic Games Store,0
2,epic_860a4feefe2a4f48905ae86c61d5db64,Lifeless Planet (Premier Edition),2015 May,Backlog,Short,Epic Games Store,0
3,epic_8935bb3e1420443a9789fe01758039a5,Alien: Isolation,2014 Oct,Tried,Short,Epic Games Store,2 Hours 22 Minutes
4,epic_8e425d507bd6473dbb927f0fea8ef4f6,Torchlight 2,2012 Sep,Backlog,Long,Epic Games Store,0
...,...,...,...,...,...,...,...
158,xboxone_1793054097,Carrion,2020 Jul,Completed,Short,XBGP,2 Hours 40 Minutes
159,xboxone_1906143168,Crusader Kings 3,2020 Sep,Tried,Infinite,XBGP,3 Hours 11 Minutes
160,xboxone_1985701699,Forza Horizon 4,2018 Oct,Completed,Infinite,XBGP,28 Hours 21 Minutes
161,xboxone_2105342780,Grounded,2020 Jul,Tried,Infinite,XBGP,1 Hours 17 Minutes
